**ANN Model for the Classification**

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [ ]:
# Loading dataset
df = pd.read_csv("../feature_engineered_with_pca_optimal.csv", header=0)

y = df.iloc[:, 53:57]                   # one-hot encoded class labels
X_r = df.iloc[:,:25]                    # raw geometry data
X_sff = df.iloc[:, 25:35]               # SFF data
X_PCA = df.iloc[:,36:53]                # PCA reduced data
print(y.shape)
print(X_PCA.shape)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_r, y, test_size=0.2, stratify= y, random_state=42, shuffle=True)

# Scaling input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**ANN Model**

In [ ]:
# Building the Model
model = Sequential()
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training Model with EarlyStopping constrain
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Evaluation of Model
y_pred_probs = model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test.values, axis=1)

test_accuracy = np.mean(y_pred_classes == y_true_classes) * 100
print(f"\nModel Test Accuracy: {test_accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4))

In [ ]:
# Confusion Matrix and Graph Plotting
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0-1500","1501-2500","2500-3500",">3500"], yticklabels=["0-1500","1501-2500","2500-3500",">3500"])
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show(block=False)

fig, ax1 = plt.subplots(figsize=(8, 5))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:red')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:red')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.plot(history.history['accuracy'], label='Training Accuracy', color='tab:blue')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='tab:cyan')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='center right')

ymin, ymax = ax1.get_ylim()
ax2.set_ylim(ymin, ymax)

plt.title('Training & Validation Loss + Accuracy')
plt.grid()
plt.show()
plt.close()

**Model with SFF**

In [ ]:
# Train-test split
X_train_sff, X_test_sff, y_train, y_test = train_test_split(X_sff, y, test_size=0.2, stratify= y, random_state=42, shuffle=True)

# Scaling input data
scaler = StandardScaler()
X_train_sff = scaler.fit_transform(X_train_sff)
X_test_sff = scaler.transform(X_test_sff)

# Building the Model
model = Sequential()
model.add(keras.Input(shape=(X_train_sff.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training Model with EarlyStopping constrain
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train_sff, y_train,
    validation_split=0.1,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Evaluation of Model
y_pred_probs = model.predict(X_test_sff)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test.values, axis=1)

test_accuracy = np.mean(y_pred_classes == y_true_classes) * 100
print(f"\nModel Test Accuracy: {test_accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4))

# Confusion Matrix and Graph Plotting
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0-1500","1501-2500","2500-3500",">3500"], yticklabels=["0-1500","1501-2500","2500-3500",">3500"])
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show(block=False)

fig, ax1 = plt.subplots(figsize=(8, 5))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:red')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:red')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.plot(history.history['accuracy'], label='Training Accuracy', color='tab:blue')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='tab:cyan')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='upper right')

ymin, ymax = ax1.get_ylim()
ax2.set_ylim(ymin, ymax)

plt.title('Training & Validation Loss + Accuracy')
plt.grid()
plt.show()
plt.close()

**Model with PCA reduced Raw and SFF features**

In [ ]:
# Train-test split
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_PCA, y, test_size=0.2, stratify= y, random_state=42, shuffle=True)

# Scaling input data
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

# Building the Model
model = Sequential()
model.add(keras.Input(shape=(X_train_pca.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training Model with EarlyStopping constrain
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train_pca, y_train,
    validation_split=0.1,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Evaluation of Model
y_pred_probs = model.predict(X_test_pca)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test.values, axis=1)

test_accuracy = np.mean(y_pred_classes == y_true_classes) * 100
print(f"\nModel Test Accuracy: {test_accuracy:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4))

# Confusion Matrix and Graph Plotting
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0-1500","1501-2500","2500-3500",">3500"], yticklabels=["0-1500","1501-2500","2500-3500",">3500"])
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show(block=False)

fig, ax1 = plt.subplots(figsize=(8, 5))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:red')
ax1.plot(history.history['loss'], label='Training Loss', color='tab:red')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.plot(history.history['accuracy'], label='Training Accuracy', color='tab:blue')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='tab:cyan')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='center right')

ymin, ymax = ax1.get_ylim()
ax2.set_ylim(ymin, ymax)

plt.title('Training & Validation Loss + Accuracy')
plt.grid()
plt.show()
plt.close()